In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [25]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [26]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://beritajatim.com/kanal/politik-pemerintahan/page/{page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3')
    
    page_links = []
    for article in articles:
        link_element = article.find('a')
        if link_element is not None:
            link = link_element.get('href')
            if link is not None:
                page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [27]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 29 links from page 1
Total Links: 29


In [28]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "date-time"})
            if author_elem:
                author_text = author_elem.find('p')
                author_text = author_text.text
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"itemprop": "datePublished"})
            if date_elem:
                date_text = date_elem.text
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('span', {"class": "kanal"})
            if category_elements:
                category_text= category_elements.text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jatim',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [29]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [30]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

29


,title,author,category,date,content,region,link
0,LaNyalla Ajak Forum Doktor Kawal Proses Restor...,Reporter : Ahmad Baiquni,Politik Pemerintahan,"Rabu, 27 September 2023, 12:26 WIB",Jakarta (beritajatim.com) – Ketua Dewan Perwak...,jatim,https://beritajatim.com/politik-pemerintahan/l...
1,"Bonceng Tiga, Pelajar Sampang Meninggal Tabrak...",Reporter : Zamachsari,Peristiwa,"Selasa, 26 September 2023, 16:03 WIB",Sampang (beritajatim.com) – Satu pelajar menin...,jatim,https://beritajatim.com/peristiwa/bonceng-tiga...
2,Sidang Paripurna DPRD Jember Jadi Arena Adu Pa...,Reporter : Oryza A. Wirawan,Politik Pemerintahan,"Selasa, 26 September 2023, 18:19 WIB",Jember (beritajatim.com) – Sidang paripurna pe...,jatim,https://beritajatim.com/politik-pemerintahan/s...
3,Lurah se-Blitar Sebut Ada Pungli Soal Tanah Ek...,Reporter : Winanto,Politik Pemerintahan,"Rabu, 27 September 2023, 13:17 WIB",Blitar (beritajatim.com) – Komisi I DPRD Kabup...,jatim,https://beritajatim.com/politik-pemerintahan/k...
4,"DK4 Somasi Komunitas Artefak Nusantara, Ini Ma...",Reporter : Nanang Masyhari,Hukum & Kriminal,"Selasa, 26 September 2023, 23:33 WIB",Kediri (beritajatim.com) – Dewan Kesenian dan ...,jatim,https://beritajatim.com/hukum-kriminal/dk4-som...


In [31]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/berita_jatim_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')